# Feature engineering examples

This notebook contains some examples of feature engineering using SAM.

In [ ]:
import pandas as pd

data = pd.read_parquet('../data/rainbow_beach.parquet')

data.head()

## Simple feature engineering for timeseries data

The class `sam.feature_engineering.SimpleFeatureEngineering` is used to create common features for timeseries data: rolling features and time components.

A rolling feature can be parameterized by a tuple of the form `(column_name, method, rolling_window)`, where `column_name` is the name of the column to be used as the time series, `method` is the type of rolling feature (e.g. "mean", "lag", "max"), and `rolling_window` is the size of the rolling window.

A time component can be either be descibed by a dummy variable (one hot encoding) or a cyclic variable (sin/cos). To parameterize a cyclic variable, a tuple of the form `(period, component_type)` is used.

In [ ]:
from sam.feature_engineering import SimpleFeatureEngineer


sfe = SimpleFeatureEngineer(
    rolling_features=[
        ("water_temperature", "mean", "1D"),
        ("water_temperature", "mean", "2D"),
        ("turbidity", "max", "1D"),
        ("turbidity", "max", "2D"),
    ],
    time_features=[
        ("hour_of_day", "cyclical"),
        ("day_of_week", "cyclical"),
    ],
    time_col="TIME",  # leave as None if using a time index
)

sfe.fit_transform(data)

## Custom feature engineering function

If you want more freedom and customize your feature engineering, you can use `sam.feature_engineering.FeatureEngineering` to create your own feature engineering transformer from a feature engineering function. This class provides methods to make sure the interface is compatible with sam models.


In [ ]:
from sam.feature_engineering import FeatureEngineer

def my_feature_engineering(X, y=None):
    """Don't forget documentation
    """
    X_out = X.copy()
    X_out = X_out[["battery_life", "water_temperature", "turbidity"]]
    X_out['my_feature'] = X_out['water_temperature'].rolling(window=24).mean().pow(2)
    return X_out

my_fe = FeatureEngineer(my_feature_engineering)

my_fe.fit_transform(data)

## Customized feature engineering class

If a single function does not fit your needs, you can create your own feature engineering class. By creating a subclass of `sam.feature_engineering.BaseFeatureEngineer`, you can implement your own feature engineering function. You only need to implement the `feature_engineer_` method. If you want to fit certain parameters, you can implement the `fit` method as well. Check the current implementation of `BaseFeatureEngineer` or `SimpleFeatureEngineer` for an example.

